<a href="https://colab.research.google.com/github/zeqkik/LLM-Fine-Tuning/blob/main/fine_tuning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
if torch.cuda.is_available():
    print("GPU está disponível! Usando:", torch.cuda.get_device_name(0))
    device = "cuda"
else:
    print("Nenhuma GPU disponível. Usando CPU. O treinamento será lento.")
    device = "cpu"

GPU está disponível! Usando: Tesla T4


In [ ]:
from huggingface_hub import login
login() # Siga as instruções para colar seu token de acesso.

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [11]:


# Nome do modelo Gemma 2B
model_id = "google/gemma-2b"

# Carrega o tokenizador
# Carrega o tokenizador
tokenizer_gemma = AutoTokenizer.from_pretrained(model_id)

# *** MODIFICAÇÃO CHAVE AQUI: Definir o pad_token se não existir ***
# Isso é VITAL para evitar loops na função .generate()
if tokenizer_gemma.pad_token is None:
    tokenizer_gemma.pad_token = tokenizer_gemma.eos_token
    print(f"pad_token definido como eos_token: {tokenizer_gemma.pad_token}")




In [ ]:
# Carrega o modelo
# load_in_4bit=True faz o carregamento quantizado para economizar memória da GPU
# torch_dtype=torch.bfloat16 (ou torch.float16) para melhor desempenho em GPUs compatíveis
# device_map="auto" distribui o modelo automaticamente pela GPU(s)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # Ou torch.float16 se bfloat16 não for suportado pela GPU
    device_map="auto",
    load_in_4bit=True # Para economizar memória da GPU
)


In [6]:
text = "O que você sabe sobre o jogo UNO?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)


In [ ]:
model_id = "google/gemma-7b-it"

tokenizer_gemma = AutoTokenizer.from_pretrained(model_id)

model_gemma = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
)


In [55]:
def generate_response(prompt_text):
  input_ids = tokenizer_gemma(prompt_text, return_tensors="pt").to(model_gemma.device)

  outputs = model_gemma.generate(
        **input_ids,
        max_new_tokens=50,
    )

  response = tokenizer_gemma.decode(outputs[0], skip_special_tokens=True)
  return response


In [56]:
prompt_1 = """
Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?
"""

In [54]:
print(generate_response(prompt_1))


Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?
A. Azul 3
B. Verde 5
C. Vermelho 8
D. +2 Amarelo

A resposta para a questão é a carta Vermelho 8.

A carta no descarte é Vermelho 


In [59]:
prompt_2 = """
Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
"""

In [60]:
print(generate_response(prompt_2))


Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
**Resposta:** A carta que você podem jogar é a carta Verde 2, pois a regra do Veretchup + 3 exige que o jogador jogue uma carta de mesmo nome que foi lançada no início do jogo.


In [61]:
prompt_3 = """
Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
"""

In [62]:
print(generate_response(prompt_3))


Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
**Resposta:** O Verde 2.

**Explicação:**

No momento em que a carta Azul foi descartada, a regra do "Uno" exige que o próximo carta seja de mesmo número ou de mesma cor. Na mão do jogador,
